# **Projeto de Business Intelligence e Analytics: Análise da Produção de Soja no Paraná**

### **Introdução:**

A soja é uma das commodities mais importantes para a economia brasileira, com o Paraná se destacando como o segundo maior produtor nacional. Diante da relevância desse setor, a capacidade de prever a produção futura é um diferencial estratégico para o planejamento logístico, de mercado e de recursos.

Este notebook faz parte da **Fase 2 do projeto em BI e Analytics** e apresenta a solução completa para a análise e previsão da produção de soja, utilizando a abordagem de Machine Learning.

### **Componentes Principais:**

*   **Análise Exploratória**: Investigação dos dados históricos de produção, área plantada, área colhida e rendimento.
* **Modelagem Preditiva**: Implementação e comparação de três modelos de previsão: Random Forest, ARIMA e Holt-Winters.
* **Avaliação de Desempenho**: Avaliação rigorosa dos modelos usando métricas como MAE, RMSE, R² e MAPE.
 **Geração de Previsões**: Utilização do modelo de melhor desempenho para gerar previsões mensais e anuais para os próximos anos.

### **Conclusão da Análise:**

Após a avaliação dos modelos, o **Random Forest** foi identificado como a melhor solução para este problema de previsão, apresentando um MAPE de apenas 1.25% e um R² de 0.97. Os resultados detalhados e as previsões futuras podem ser encontrados nas seções a seguir.

## 1. Instalação de Bibliotecas


Para garantir que o ambiente de execução contenha todas as ferramentas necessárias para a análise, instalamos as seguintes bibliotecas:

* `numpy<2`: A biblioteca `numpy` é fundamental para operações numéricas em Python. A versão 1.x é instalada para garantir compatibilidade com a biblioteca `pmdarima`, que será usada para o modelo ARIMA.

* `pmdarima`: Esta biblioteca é uma extensão da `scikit-learn` e facilita a busca automática dos melhores parâmetros (`p`, `d`, `q`) para o modelo ARIMA.

In [ ]:
# !pip install "numpy<2"
# !pip install pmdarima

In [ ]:
import pandas as pd
import pandas as pd
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing


## 2. Carregamento  dos Dados

Nesta etapa, os dados históricos sobre a produção de soja no Paraná, provenientes da **base de dados do LSPA/IBGE**, são carregados e unificados. Cada arquivo CSV representa uma variável-chave para o projeto:

* `area_plantada_soja_PR_2020-2024.csv`: Dados sobre a área plantada em hectares.

* `area_colhida_soja_PR_2020-2024.csv`: Dados sobre a área colhida em hectares.

* `producao_soja_PR_2020-2024.csv`: Dados da produção total em toneladas.

* `rendimento_soja_PR_2020-2024.csv`: Dados do rendimento médio por hectare.

Os arquivos foram lidos utilizando o separador **ponto e vírgula** (`sep=';'`).

In [ ]:
df_plantada= pd.read_csv('area_plantada_soja_PR_2020-2024.csv', sep=';')
df_colhida= pd.read_csv('area_colhida_soja_PR_2020-2024.csv', sep=';')
df_producao= pd.read_csv('producao_soja_PR_2020-2024.csv', sep=';')
df_rendimento= pd.read_csv('rendimento_soja_PR_2020-2024.csv', sep=';')

In [ ]:
df_plantada.head()

In [ ]:
df_colhida.head()

In [ ]:
df_producao.head()

In [ ]:
df_rendimento.head()

## 3. Limpeza, Transformação e Unificação dos Dados

Nesta etapa, os DataFrames individuais são limpos e transformados para garantir que estejam prontos para a análise. O objetivo é remover dados desnecessários, padronizar o formato e unificar todas as informações em um único DataFrame coeso.

A seguir, a função `apagar_linhas` é usada para remover linhas de cabeçalho e rodapé que podem estar presentes nos arquivos, garantindo que apenas os dados válidos sejam mantidos.

In [ ]:
# Define uma função para remover linhas indesejadas dos DataFrames
def apagar_linhas(df):
    # Remove a primeira linha (index 0)
    df = df.iloc[1:].reset_index(drop=True)

    # Se a última linha tiver todos os valores NaN, remove
    if df.tail(1).isna().all(axis=1).iloc[0]:
        df = df.iloc[:-1].reset_index(drop=True)

    return df

In [ ]:
# Aplica a função de limpeza a cada um dos DataFrames
df_plantada= apagar_linhas(df_plantada)
df_colhida= apagar_linhas(df_colhida)
df_producao= apagar_linhas(df_producao)
df_rendimento= apagar_linhas(df_rendimento)

In [ ]:
df_plantada.head()

,janeiro 2020,fevereiro 2020,março 2020,abril 2020,maio 2020,junho 2020,julho 2020,agosto 2020,setembro 2020,outubro 2020,...,março 2024,abril 2024,maio 2024,junho 2024,julho 2024,agosto 2024,setembro 2024,outubro 2024,novembro 2024,dezembro 2024
0,5500482,5501182,5504222,5505872,5503500,5503000,5503200,5505000,5511200,5512600,...,5815700,5814700,5819100,5819700,5824800,5835400,5835100,5835100,5835000,5835000


In [ ]:
df_colhida.head()

In [ ]:
df_producao.head()

In [ ]:
df_rendimento.head()

Em seguida, os dados, que estavam em formato largo (uma coluna por mês), são transformados para o formato longo usando a função `pd.melt`. Isso cria duas colunas: uma para a data e outra para o valor correspondente.

In [ ]:
# Converte cada DataFrame do formato "largo" para o formato "longo"
df_plantada = pd.melt(df_plantada, var_name='data', value_name='plantada')
df_colhida = pd.melt(df_colhida, var_name='data', value_name='colhida')
df_producao = pd.melt(df_producao, var_name='data', value_name='producao')
df_rendimento = pd.melt(df_rendimento, var_name='data', value_name='rendimento')

Para facilitar a análise de séries temporais, as datas são formatadas para um padrão consistente (`jan/20`, `fev/20`, `etc.`). Isso é feito através de um dicionário de mapeamento e de uma função dedicada.

In [ ]:
# Mapeia os meses de português para uma abreviação curta
MAPA_MESES = {
    "janeiro": "jan", "fevereiro": "fev", "março": "mar",
    "abril": "abr", "maio": "mai", "junho": "jun",
    "julho": "jul", "agosto": "ago", "setembro": "set",
    "outubro": "out", "novembro": "nov", "dezembro": "dez"
}

def formatar_data(df: pd.DataFrame, col_data="data") -> pd.DataFrame:
    """
    Converte 'janeiro 2020' → 'jan/20'.
    """
    # separa mês e ano
    partes = df[col_data].str.split(" ", n=1, expand=True)
    mes = partes[0].str.lower()
    ano = partes[1]

    # converte ano para 2 dígitos
    ano2 = ano.str[-2:]

    # aplica abreviação
    mes_abrev = mes.map(MAPA_MESES)

    # monta novo formato
    df[col_data] = mes_abrev + "/" + ano2
    return df

In [ ]:
# Aplica a função de formatação de data a cada DataFrame
df_plantada = formatar_data(df_plantada, "data")
df_colhida = formatar_data(df_colhida, "data")
df_producao = formatar_data(df_producao, "data")

In [ ]:
df_plantada.head()

In [ ]:
df_colhida.head()

In [ ]:
df_producao.head()

Finalmente, os quatro DataFrames são mesclados em um único DataFrame chamado `df_soja`. Em seguida, as colunas de data e os tipos de dados são ajustados para garantir que estejam no formato correto para a análise de Machine Learning.

In [ ]:
# Unifica os DataFrames em um único conjunto de dados completo
df_soja = (df_plantada
           .merge(df_colhida, on='data')
           .merge(df_producao, on='data')
           .merge(df_rendimento, on='data'))

In [ ]:
# Mapeia as abreviações de mês em português para inglês para a conversão de data
meses_pt_en = {
    'jan': 'Jan', 'fev': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'mai': 'May',
    'jun': 'Jun', 'jul': 'Jul', 'ago': 'Aug', 'set': 'Sep', 'out': 'Oct',
    'nov': 'Nov', 'dez': 'Dec'
}

# Cria uma nova coluna 'data_datetime' e extrai 'mes' e 'ano'
df_soja['data_datetime'] = df_soja['data'].str.split('/').str[0].map(meses_pt_en) + ' ' + df_soja['data'].str.split('/').str[1]
df_soja['data_datetime'] = pd.to_datetime(df_soja['data_datetime'], format='%b %y')
df_soja['mes'] = df_soja['data_datetime'].dt.strftime('%B')
df_soja['ano'] = df_soja['data_datetime'].dt.strftime('%Y')
df_soja['data'] = df_soja['data_datetime'].dt.strftime('%Y-%m')


In [ ]:
# Seleciona as colunas finais
df_soja = df_soja[['data', 'mes', 'ano', 'plantada', 'colhida', 'producao', 'rendimento']]

# converter a coluna 'data' para o tipo datetime e as outras para numérico
df_soja['data'] = pd.to_datetime(df_soja['data'])
df_soja["plantada"] = pd.to_numeric(df_soja["plantada"])
df_soja["colhida"] = pd.to_numeric(df_soja["colhida"])
df_soja["producao"] = pd.to_numeric(df_soja["producao"])
df_soja["rendimento"] = pd.to_numeric(df_soja["rendimento"])

In [ ]:
df_soja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   data        60 non-null     datetime64[ns]
 1   mes         60 non-null     object        
 2   ano         60 non-null     object        
 3   plantada    60 non-null     int64         
 4   colhida     60 non-null     int64         
 5   producao    60 non-null     int64         
 6   rendimento  60 non-null     int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 3.4+ KB


In [ ]:
# Salva o DataFrame final tratado em um novo arquivo CSV
df_soja.to_csv('df_soja_tratado.csv', index=False)
print("\nDados tradados salvos em 'df_soja_tratado.csv'")


Dados tradados salvos em 'df_soja_tratado.csv'


# RANDOM FLOREST

In [ ]:
# 1. Preparar os dados
# Definir a variável alvo (y) e as features (X)
X = df_soja[['plantada', 'colhida', 'rendimento', 'ano']] # Features
y = df_soja['producao'] # Variável alvo

In [ ]:
# 2. Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 3. Treinar o modelo
# Instanciar e treinar o modelo Random Forest
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# 4. Fazer previsões no conjunto de teste
y_pred = model_rf.predict(X_test)

In [ ]:
# 5. Avaliar o desempenho do modelo
mae_rf = mean_absolute_error(y_test, y_pred)
mse_rf = mean_squared_error(y_test, y_pred)
rmse_rf = np.sqrt(mse_rf)
mape_rf = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2_rf = r2_score(y_test, y_pred)

In [ ]:
# Imprimir os resultados
print("\nResultados de desempenho do modelo ARIMA:")
print("-" * 40)
print(f"1. Erro Médio Absoluto (MAE): {mae_rf:.2f}")
print(f"2. Erro Quadrático Médio (MSE): {mse_rf:.2f}")
print(f"3. Raiz do Erro Quadrático Médio (RMSE): {rmse_rf:.2f}")
print(f"4. Coeficiente de Determinação (R-quadrado): {r2_rf:.2f}")
print("-" * 40)
print(f"Erro Percentual Médio Absoluto (MAPE): {mape_rf:.2f}%")


Resultados de desempenho do modelo ARIMA:
----------------------------------------
1. Erro Médio Absoluto (MAE): 251201.27
2. Erro Quadrático Médio (MSE): 211586083378.41
3. Raiz do Erro Quadrático Médio (RMSE): 459984.87
4. Coeficiente de Determinação (R-quadrado): 0.97
----------------------------------------
Erro Percentual Médio Absoluto (MAPE): 1.25%


# ARIMA

In [ ]:
# 1. Preparar os dados
# Usaremos apenas a coluna 'producao' para o modelo ARIMA
producao_arima = df_soja['producao']

In [ ]:
# 2. Dividir os dados em conjuntos de treinamento e teste
# O modelo ARIMA não usa 'train_test_split' da mesma forma, pois a ordem dos dados é importante.
# Vamos usar os últimos 12 meses (1 ano) para teste.
train_data = producao_arima[:-12]
test_data = producao_arima[-12:]

In [ ]:
# 3. Identificar os parâmetros ARIMA usando auto_arima
# Esta função encontra a melhor combinação de p, d, q para o modelo
print("Buscando os melhores parâmetros para o modelo ARIMA...")
model_fit = auto_arima(train_data, seasonal=True, m=12, trace=True,
                       error_action='ignore', suppress_warnings=True)

print("Melhores parâmetros encontrados:", model_fit.order)

Buscando os melhores parâmetros para o modelo ARIMA...
Performing stepwise search to minimize aic


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12

 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=1509.729, Time=0.37 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=1597.439, Time=0.01 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=1502.923, Time=0.10 sec


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=1570.415, Time=0.10 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=1748.050, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=1514.480, Time=0.02 sec


...

 ARIMA(1,0,1)(2,0,0)[12] intercept   : AIC=1503.812, Time=0.24 sec
 ARIMA(0,0,1)(2,0,0)[12] intercept   : AIC=1575.645, Time=0.17 sec


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,1)(2,0,0)[12] intercept   : AIC=1504.155, Time=0.92 sec


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,0)(2,0,0)[12]             : AIC=inf, Time=0.28 sec

Best model:  ARIMA(1,0,0)(2,0,0)[12] intercept
Total fit time: 3.390 seconds
Melhores parâmetros encontrados: (1, 0, 0)


In [ ]:
# 4. Fazer previsões
# Fazer previsões para o período de teste
predictions = model_fit.predict(n_periods=len(test_data))


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
# Criar um DataFrame para facilitar a avaliação
predictions_df = pd.DataFrame(predictions, index=test_data.index)
predictions_df.columns = ['producao_predita']

In [ ]:
# 5. Avaliar o desempenho do modelo
# Calcular as métricas
mae_arima = mean_absolute_error(test_data, predictions)
mse_arima = mean_squared_error(test_data, predictions)
rmse_arima = np.sqrt(mse_arima)
r2_arima = r2_score(test_data, predictions)
mape_arima = np.mean(np.abs((test_data - predictions) / test_data)) * 100

In [ ]:
# Imprimir os resultados
print("\nResultados de desempenho do modelo ARIMA:")
print("-" * 40)
print(f"1. Erro Médio Absoluto (MAE): {mae_arima:.2f}")
print(f"2. Erro Quadrático Médio (MSE): {mse_arima:.2f}")
print(f"3. Raiz do Erro Quadrático Médio (RMSE): {rmse_arima:.2f}")
print(f"4. Coeficiente de Determinação (R-quadrado): {r2_arima:.2f}")
print("-" * 40)
print(f"Erro Percentual Médio Absoluto (MAPE): {mape_arima:.2f}%")


Resultados de desempenho do modelo ARIMA:
----------------------------------------
1. Erro Médio Absoluto (MAE): 1036003.78
2. Erro Quadrático Médio (MSE): 1130222267060.13
3. Raiz do Erro Quadrático Médio (RMSE): 1063119.12
4. Coeficiente de Determinação (R-quadrado): -18.41
----------------------------------------
Erro Percentual Médio Absoluto (MAPE): 5.55%


#  Holt-Winters


In [ ]:
# 1. Preparar os dados
# Usar apenas a coluna 'producao' para o modelo de série temporal
producao_hw = df_soja['producao']

In [ ]:
# 2. Dividir os dados em conjuntos de treinamento e teste
# Usar os últimos 12 meses (1 ano) para o conjunto de teste, assim como no modelo ARIMA.
train_data = producao_hw[:-12]
test_data = producao_hw[-12:]

In [ ]:
# 3. Treinar o modelo Holt-Winters
# Instanciar e treinar o modelo de suavização exponencial.
# 'trend="add"' indica uma tendência aditiva (constante).
# 'seasonal="add"' indica uma sazonalidade aditiva.
# 'seasonal_periods=12' indica que o ciclo sazonal é de 12 meses.
model_hw = ExponentialSmoothing(train_data, trend='add', seasonal='add', seasonal_periods=12,
                                 initialization_method="estimated").fit()

In [ ]:
# 4. Fazer previsões
# Fazer previsões para o período de teste.
predictions = model_hw.forecast(steps=len(test_data))

In [ ]:
# 5. Avaliar o desempenho do modelo
# Calcular as métricas
mae_hw = mean_absolute_error(test_data, predictions)
mse_hw = mean_squared_error(test_data, predictions)
rmse_hw = np.sqrt(mse_hw)
r2_hw = r2_score(test_data, predictions)
mape_hw = np.mean(np.abs((test_data - predictions) / test_data)) * 100

In [ ]:
# Imprimir os resultados
print("Resultados de desempenho do modelo Holt-Winters:")
print("-" * 40)
print(f"1. Erro Médio Absoluto (MAE): {mae_hw:.2f}")
print(f"2. Erro Quadrático Médio (MSE): {mse_hw:.2f}")
print(f"3. Raiz do Erro Quadrático Médio (RMSE): {rmse_hw:.2f}")
print(f"4. Coeficiente de Determinação (R-quadrado): {r2_hw:.2f}")
print("-" * 40)
print(f"Erro Percentual Médio Absoluto (MAPE): {mape_hw:.2f}%")

Resultados de desempenho do modelo Holt-Winters:
----------------------------------------
1. Erro Médio Absoluto (MAE): 4006709.04
2. Erro Quadrático Médio (MSE): 16105300709612.13
3. Raiz do Erro Quadrático Médio (RMSE): 4013141.00
4. Coeficiente de Determinação (R-quadrado): -275.53
----------------------------------------
Erro Percentual Médio Absoluto (MAPE): 21.52%


# Analises Final

In [ ]:
# Criar e imprimir a tabela de comparação
# -----------------------------------------------------------
results = {
    'Modelo': ['Random Forest', 'ARIMA', 'Holt-Winters'],
    'MAE': [mae_rf, mae_arima, mae_hw],
    'RMSE': [rmse_rf, rmse_arima, rmse_hw],
    'R-squared': [r2_rf, r2_arima, r2_hw],
    'MAPE (%)': [mape_rf, mape_arima, mape_hw]
}
df_results = pd.DataFrame(results).set_index('Modelo')

In [ ]:
print("Tabela de Comparação de Desempenho dos Modelos:")
print("---")
print(df_results.to_markdown(floatfmt=".2f"))

Tabela de Comparação de Desempenho dos Modelos:
---
| Modelo        |        MAE |       RMSE |   R-squared |   MAPE (%) |
|:--------------|-----------:|-----------:|------------:|-----------:|
| Random Forest |  251201.27 |  459984.87 |        0.97 |       1.25 |
| ARIMA         | 1036003.78 | 1063119.12 |      -18.41 |       5.55 |
| Holt-Winters  | 4006709.04 | 4013141.00 |     -275.53 |      21.52 |


# PREVISAO PARA OS PROXIMOS ANOS

In [ ]:
# 1. Calcular a média das features para cada mês do último ano completo
# Isso nos permite criar previsões mais realistas, mantendo o padrão mensal.
df_soja['data'] = pd.to_datetime(df_soja['data'])
df_soja['mes'] = df_soja['data'].dt.month

medias_por_mes = df_soja.tail(12).groupby('mes')[['plantada', 'colhida', 'rendimento']].mean()

# 2. Criar um DataFrame para as previsões futuras por mês e ano
previsoes_futuras = []
anos_futuros = [2025, 2026, 2027]
meses = list(range(1, 13))

for ano in anos_futuros:
    for mes in meses:
        # Obter os valores médios das features para o mês específico
        # Se um mês não estiver presente nos dados, usar a média geral
        if mes in medias_por_mes.index:
            plantada = medias_por_mes.loc[mes, 'plantada']
            colhida = medias_por_mes.loc[mes, 'colhida']
            rendimento = medias_por_mes.loc[mes, 'rendimento']
        else:
            plantada = df_soja['plantada'].mean()
            colhida = df_soja['colhida'].mean()
            rendimento = df_soja['rendimento'].mean()

        # Criar os dados de entrada para o modelo
        dados_modelo = pd.DataFrame([[plantada, colhida, rendimento, ano]],
                                    columns=['plantada', 'colhida', 'rendimento', 'ano'])

        # Fazer a previsão
        previsao = model_rf.predict(dados_modelo)

        previsoes_futuras.append({
            'Ano': ano,
            'Mês': mes,
            'Produção Prevista (toneladas)': previsao[0]
        })

# Criar um DataFrame a partir dos resultados
df_previsoes_mensais = pd.DataFrame(previsoes_futuras)

print("Previsões de Produção de Soja por Mês (2025-2027):")
print(df_previsoes_mensais.to_markdown(index=False, numalign="left", stralign="left"))


Previsões de Produção de Soja por Mês (2025-2027):
| Ano   | Mês   | Produção Prevista (toneladas)   |
|:------|:------|:--------------------------------|
| 2025  | 1     | 1.85854e+07                     |
| 2025  | 2     | 1.8394e+07                      |
| 2025  | 3     | 1.84582e+07                     |
| 2025  | 4     | 1.84717e+07                     |
| 2025  | 5     | 1.85212e+07                     |
| 2025  | 6     | 1.85644e+07                     |
| 2025  | 7     | 1.85883e+07                     |
| 2025  | 8     | 1.8665e+07                      |
| 2025  | 9     | 1.86408e+07                     |
| 2025  | 10    | 1.86408e+07                     |
| 2025  | 11    | 1.86428e+07                     |
| 2025  | 12    | 1.86428e+07                     |
| 2026  | 1     | 1.85854e+07                     |
| 2026  | 2     | 1.8394e+07                      |
| 2026  | 3     | 1.84582e+07                     |
| 2026  | 4     | 1.84717e+07                     |
| 2026  | 5  

In [ ]:
# Salvar as previsões em um arquivo CSV
df_previsoes_mensais.to_csv('previsoes_soja_mensais.csv', index=False)
print("\nPrevisões salvas em 'previsoes_soja_mensais.csv'")


Previsões salvas em 'previsoes_soja_mensais.csv'
